In [ ]:
import pandas as pd
from pathlib import Path
import statsmodels
import statsmodels.api as sm
import statsmodels.formula.api as smf
import numpy as np
import plotly.express as px
import yaml
with open("config.yaml", "r") as fh:
    config_dict = yaml.safe_load(fh)['default']
from sklearn.metrics import precision_score, recall_score, confusion_matrix

In [ ]:
%ls /nfs/nas22/fs2202/biol_micro_bioinf_nccr/hardt/nguyenb/tnseq/scratch/03_23_transcriptomics/modeling

In [ ]:
root = Path(config_dict["root"])
synth_dir = root/config_dict["synth_dir"]
sample_data_file = root/config_dict["sample_data_file"]
spikes = pd.read_table(synth_dir/"synth_mgx_mtx/true-exp.mtx_spiked.tsv", header=None, names=['feature', 'positive'])
spikes.shape

# Saving synthetic data in the format accepted by deseq2. 
- Added 1 because with such low 'depth' DESeq2 fails.

In [ ]:
 
dsdf = pd.read_table(synth_dir/"synth_mgx_mtx/true-exp.mtx_abunds.tsv", index_col=0).T.reset_index().rename(columns={'index': 'sample_id'})
ds_meta = dsdf[['sample_id', 'Phenotype']]
dsdf = dsdf.drop(columns=['Phenotype', 'SeqDepth']).set_index('sample_id').T.reset_index().rename(columns={'#':'ID'}).set_index('ID')
dsdf = dsdf + 1
#ds_meta.to_csv(synth_dir/'true-exp-meta.csv', index=False)
#dsdf.reset_index().to_csv(synth_dir/"true-exp-counts.csv", index=False)

# DESeq results on all samples with 1 added to all counts

In [ ]:

# Analysis with global normalisation
deres = pd.read_csv(synth_dir/"2023-07-31_true-exp-deseq-1_vs_0_l0a0.01_results.csv")
deres = deres.merge(spikes, left_on='ID', right_on='feature', how='outer')
# Analysis with taxon specific normalisation
deres_tx = pd.read_csv(synth_dir/"2023-07-31_true-exp-deseq-within-taxon-1_vs_0_l0a0.01_results.csv")
deres_tx = deres_tx.merge(spikes, left_on='ID', right_on='feature', how='outer')

In [ ]:
fdr = 0.01
predicted = (deres.padj < fdr).astype(int)
actual = (deres.positive.notnull()).astype(int)
predicted_tx = (deres_tx.padj < fdr).astype(int)
actual_tx = (deres_tx.positive.notnull()).astype(int)
deseq_all = [precision_score(actual, predicted), recall_score(actual, predicted)] + list(confusion_matrix(actual, predicted).ravel().astype(int))
deseq_tx = [precision_score(actual_tx, predicted_tx), recall_score(actual_tx, predicted_tx)] + list(confusion_matrix(actual_tx, predicted_tx).ravel().astype(int))
labels = ['Precision', 'Recall', 'TN', 'FP', 'FN', 'TP']
bench_df = pd.DataFrame([deseq_all, deseq_tx], columns=labels, index = ['Global', 'Taxon']).apply(round,ndigits=2)

In [ ]:
bench_df

# Same analysis with linear models

In [ ]:
df = pd.read_table(synth_dir/"synth_mgx_mtx/true-exp.mtx_abunds.tsv", index_col=0).T.reset_index().rename(columns={'index': 'sample_id'})
df = df.melt(id_vars=["sample_id", "Phenotype", "SeqDepth"], var_name="gene_name", value_name='raw_cnt')
df['tss'] = df['raw_cnt']/df['SeqDepth']*1e6
df['bug'] = df['gene_name'].str.split("_", expand=True)[0]
df = df.merge(df.groupby(['sample_id', 'bug']).raw_cnt.sum().reset_index().rename(columns={'raw_cnt':'bug_cnt'}), on=['sample_id', 'bug'], how='left')
df['within_bug'] = df['raw_cnt']/df['bug_cnt']*1e6
df['bug_perc'] = df['bug_cnt']/df['SeqDepth']


In [ ]:
df[df.sample_id == 'SAMPLE0001'].sample(5)

In [ ]:
test_gene = "BUG0007_GROUP001780"
df[df.gene_name == test_gene]

In [ ]:
# Filter out genes with less than 100 reads across samples
mask = df.groupby(['gene_name']).raw_cnt.sum() > 100
mask2 = df[df.raw_cnt > 0].groupby('gene_name').sample_id.nunique() > 10
df_filtered = df.set_index('gene_name').loc[mask&mask2,:].reset_index()

In [ ]:
gene_df = df[df.gene_name == test_gene].copy()
def linear_on_gene(gene_df, expr_col, fixed_effects):
    pseudo_count = gene_df[gene_df[expr_col] > 0][expr_col].min()/2
    gene_df[f'{expr_col}_log'] = np.log10(gene_df[expr_col] + pseudo_count)
    #gene_df['tss_log_std'] = (gene_df['tss_log'] - gene_df['tss_log'].mean())/gene_df['tss_log'].std()
    if not gene_df.empty and any([i in gene_df.columns for i in fixed_effects]):
        formula = f"{expr_col}_log ~ {' + '.join(fixed_effects)}"
        #print(formula)
        md = smf.glm(formula=formula, data=gene_df, family=sm.families.Gaussian()).fit()
        #print(md.summary())
        return [md.params[1], md.bse[1], md.pvalues[1]]
    return []

def linear_on_df(df, expr_col, fixed_effects):
    f = df.dropna().groupby('gene_name').apply(linear_on_gene, expr_col = expr_col, fixed_effects = fixed_effects).reset_index()
    f[['coef', 'se', 'pval']] = pd.DataFrame(f[0].to_list(), index = f.index)
    f = f.drop(columns=[0])
    f['padj'] = statsmodels.stats.multitest.multipletests(f.pval.values, method='fdr_bh')[1]
    return f

In [ ]:
df.shape

In [ ]:
df_filtered.shape

In [ ]:
phenotype_tx = linear_on_df(df_filtered, 'within_bug', ['Phenotype'])

In [ ]:
phenotype_tx

In [ ]:
def get_scores(result_df, ground_truth, left_feat, right_feat, name, fdr=0.05):
    df = result_df[[left_feat, 'padj']].merge(ground_truth, left_on=left_feat, right_on=right_feat, how='outer')
    predicted_phenotype_tx = (df.padj < fdr).astype(int)
    actual_phenotype_tx = (df.positive.notnull()).astype(int)
    return  pd.Series([precision_score(actual_phenotype_tx, predicted_phenotype_tx), 
                       recall_score(actual_phenotype_tx, predicted_phenotype_tx)] + list(confusion_matrix(actual_phenotype_tx, predicted_phenotype_tx).ravel().astype(int)),index=['Precision', 'Recall', 'TN', 'FP', 'FN', 'TP'], name=name)


In [ ]:
get_scores(phenotype_tx, spikes, 'gene_name', 'feature', 'linear-phenotype')

In [ ]:
phenotype_bug_tx = linear_on_df(df_filtered, 'within_bug', ['Phenotype', 'bug_perc'])

In [ ]:
get_scores(phenotype_bug_tx, spikes, 'gene_name', 'feature', 'linear-bug-phenotype')

In [ ]:
phenotype_global = linear_on_df(df_filtered, 'tss', ['Phenotype'])

In [ ]:
get_scores(phenotype_global, spikes, 'gene_name', 'feature', 'linear-bug-phenotype')

In [ ]:
linear_on_gene(gene_df, 'within_bug', ['Phenotype'])

In [ ]:
m.summary()

In [ ]:
test = df_filtered[df_filtered.gene_name.isin(test_genes)]

In [ ]:
res = linear_on_df(df_filtered, 'tss', ['Phenotype'])

In [ ]:
df_filtered.groupby('bug').bug_cnt.median().sort_values()

In [ ]:
gene_df.sample(10)

In [ ]:
test

In [ ]:
res = res.merge(spikes, left_on='gene_name', right_on='feature', how='outer')

In [ ]:
#import modin.pandas as pd

In [ ]:
res2 = linear_on_df(df_filtered, 'within_bug', ['Phenotype'])

In [ ]:
res2 = res2.merge(spikes, left_on='gene_name', right_on='feature', how='outer')

In [ ]:
res2[res2.padj < 0.05].shape[0]

In [ ]:
121/2085

In [ ]:
res2[res2.padj < 0.05].isna().sum()

In [ ]:
res3 = linear_on_df(df_filtered, 'within_bug', ['Phenotype', 'bug_perc'])

In [ ]:
res4 = linear_on_df(df_filtered, 'within_bug', ['Phenotype', 'bug_perc', 'bug_perc*Phenotype'])

In [ ]:
res4 = res4.merge(spikes, left_on='gene_name', right_on='feature', how='outer')

In [ ]:
res4['gene_name'] = res4['gene_name'].fillna(res4.feature)


In [ ]:
b7 = res4[res4.gene_name.str.contains('BUG0007')].copy()

In [ ]:
b7.shape

In [ ]:
b7[(b7.positive.notnull()) & (b7.padj < 0.05)].shape

In [ ]:
res4[res4.padj < 0.05]

In [ ]:
res3 = res3.merge(spikes, left_on='gene_name', right_on='feature', how='outer')

In [ ]:
res3.dropna(subset=['gene_name'])[~res3.positive.isna()].shape

In [ ]:
2000/6818

In [ ]:
res3[res3.padj < 0.05]

In [ ]:
import statsmodels
statsmodels.stats.multitest.multipletests(f.pval.values, method='fdr_bh')[1]

In [ ]:
x = test.dropna().groupby('gene_name').apply(linear_on_gene, expr_col = 'within_bug', fixed_effects = ['Phenotype', 'bug_perc', 'Phenotype*bug_perc']).reset_index()
y = test.dropna().groupby('gene_name').apply(linear_on_gene, expr_col = 'within_bug', fixed_effects = ['Phenotype']).reset_index()

In [ ]:
' + '.join(['Phenotype', 'bug_perc', 'Phenotype*bug_perc'])

In [ ]:
x

In [ ]:
y

In [ ]:
x[0][3].summary()

In [ ]:
y[0][3].summary()

In [ ]:
linear_on_gene(test, 'within', ['Phenotype'])

In [ ]:
px.box(gene_df, x="Phenotype", y='tss_log_std', width=400, height=400)

In [ ]:
# # Run linear model
# md = smf.ols(formula="tss_log ~ Phenotype", data=test_df)
# mdf = md.fit()
# mdf.summary()

In [ ]:
# Run linear model
md = smf.glm(formula="tss_log_std ~ Phenotype", data=gene_df, family=sm.families.Gaussian())
mdf = md.fit()

In [ ]:
mdf.summary()

In [ ]:
mdf.params

In [ ]:
mdf.bse

In [ ]:
mdf.pvalues

In [ ]:
# Results from re-running mtx package
res = pd.read_table("../test_out/all_results.tsv")

In [ ]:
res = res.merge(spikes, on='feature', how='outer')

In [ ]:
res.head(10)

In [ ]:
res[(~res.positive.isna()) & (res.qval < 0.25)]

In [ ]:
# Load results for M1 model for this dataset
m1_res = pd.read_table(data_dir/"strict_filtering/true-exp_RNA/all_results.fdr_correction.tsv")


In [ ]:
m1_res[m1_res.feature.str.contains(test_gene)]

In [ ]:
m1_res.sort_values(['coef', 'qval'], ascending=False)

In [ ]:
meta_data= df[['sample_id', 'Phenotype', 'SeqDepth']]
count_data = df[['sample_id'] + [c for c in df.columns if 'BUG' in c]].set_index('sample_id').T

In [ ]:
count_data.head()

In [ ]:
meta_data.to_csv(data_dir/"true-exp.mtx_abunds_meta.csv",  index=False)
count_data.to_csv(data_dir/"true-exp.mtx_abunds_count.csv")

In [ ]:
cov_df = pd.read_table(data_dir/"true-exp.bug_abunds.tsv").iloc[2:, :]

In [ ]:
cov_df.head()

In [ ]:
cov_df.to_csv(data_dir/"true-exp.bug_abunds_edited.csv", index=False)

In [ ]:
# Results

res = pd.read_table('../test_out/all_results.tsv')

In [ ]:
res[res.qval < 0.25]

In [ ]:
spikes = pd.read_table(data_dir/"true-exp.mtx_spiked.tsv", header=None, names=['feature', 'positive'])
spikes.shape

In [ ]:
res = res.merge(spikes, on ='feature')

In [ ]:
res[(res.qval < 0.25) & (res.positive == 1)]

In [ ]:
res.shape

In [ ]:
res.positive.value_counts()

In [ ]:
pub_res = pd.read_table("/nfs/nas22/fs2202/biol_micro_bioinf_nccr/hardt/nguyenb/tnseq/scratch/03_23_transcriptomics/modeling/strict_filtering/true-exp_mtx_vs_bug/all_results.fdr_correction.tsv")

In [ ]:
res.sort_values('qval').sample(10)

In [ ]:
x = res.merge(pub_res, on='feature', suffixes=["_me", "_pub"])

In [ ]:
pub_res.head()

In [ ]:
count_data.head()

In [ ]:
cov_df.sum()

In [ ]:
# TSS within each bug 
# BUG abundances based on TX data

In [ ]:
cdf = count_data.reset_index().rename(columns={'#': 'gene_name'}).copy()
cdf['bug'] = cdf.gene_name.str.split("_", expand=True)[1]

In [ ]:
cdf.head()

In [ ]:
cdf = cdf.melt(id_vars=['gene_name', 'bug'], var_name='sample_id', value_name='cnts')

In [ ]:
cdf.head()

In [ ]:
sm = cdf.groupby(['sample_id', 'bug']).cnts.sum()

In [ ]:
bug_tx_abund = (sm/sm.groupby(level=0).transform('sum') * 100).reset_index()

In [ ]:
bug_tx_abund

In [ ]:
sm = sm.reset_index()
sm.columns = ['sample_id', 'bug', 'bug_ab']

In [ ]:
cdf = cdf.merge(sm, on=['sample_id', 'bug'])

In [ ]:
cdf['tx_cnts'] = cdf['cnts']/cdf['bug_ab']

In [ ]:
cdf.head()

In [ ]:
cdf.pivot(index='gene_name', columns='sample_id', values='tx_cnts')

In [ ]:
test= cdf[cdf.gene_name == 'BUG0007_GROUP000391']

In [ ]:
test = test[test.tx_cnts>0]

In [ ]:
test = test.merge(meta_data, on='sample_id')
test = test.merge(bug_tx_abund, on=['sample_id', 'bug'], suffixes=['_raw', '_bug'])

In [ ]:
test

In [ ]:
mod = smf.ols(formula='tx_cnts ~ Phenotype + bug_ab', data=test)

In [ ]:
res = mod.fit()

In [ ]:
res.summary()

In [ ]:
data = sm.datasets.get_rdataset("dietox", "geepack").data

In [ ]:
data